In [1]:
import numpy as np
import os
import sys
import matplotlib.pyplot as plt

import networkx as nx
import ase
import ase.io
import time
import sqlite3 as sl

import ProbAsn.utils as ut
import ProbAsn.graph as gr

In [2]:
db_file = "../db/ProbAsn.db"
db_dir = "../db/"

elems = ["H", "C", "N", "O"]

batch_size = 1000

# Create DB file

In [3]:
con = sl.connect(db_file)

In [4]:
for e1 in elems:
    
    print(f"Creating {e1}...")
    
    with con:
        data = con.execute(f"""SELECT * from {e1}""").fetchall()
    
    con2 = sl.connect(f"{db_dir}ProbAsn_{e1}.db")
    with con2:
        con2.execute(f"""
            CREATE TABLE {e1} (
                env VARCHAR(16),
                crystal VARCHAR(16),
                ind UNSIGNED INTEGER,
                shift FLOAT,
                err FLOAT,
                G2 VARCHAR(32),
                G3 VARCHAR(32),
                G4 VARCHAR(32),
                G5 VARCHAR(32),
                G6 VARCHAR(32)
            );
        """)
    
    n_batch = len(data) // batch_size + int(len(data) / batch_size > len(data) // batch_size)
    
    for ibatch in range(n_batch):
        
        pp = f"INSERT INTO {e1} (env, crystal, ind, shift, err, G2, G3, G4, G5, G6)\nVALUES\n"
    
        for env, crystal, i, cs, err, h0, h1, h2, h3, h4 in data[ibatch*batch_size:(ibatch+1)*batch_size]:
            
            pp += f"('{env}', '{crystal}', {i}, {cs}, {err}, '{h0}', '{h1}', '{h2}', '{h3}', '{h4}'),\n"

        pp = pp[:-2] + ";"

        with con2:
            con2.execute(pp)
        
    con2.commit()
    con2.close()
    
    for e2 in elems:
        
        print(f"Creating {e1}-{e2}...")
        
        with con:
            data = con.execute(f"""SELECT * from {e1}_{e2}""").fetchall()

        con2 = sl.connect(f"{db_dir}ProbAsn_{e1}-{e2}.db")
        with con2:
            con2.execute(f"""
                CREATE TABLE {e1}_{e2} (
                    env VARCHAR(16),
                    crystal VARCHAR(16),
                    ind UNSIGNED INTEGER,
                    shift FLOAT,
                    err FLOAT,
                    nei_ind UNSIGNED INTEGER,
                    nei_shift FLOAT,
                    nei_err FLOAT,
                    G2 VARCHAR(32),
                    G3 VARCHAR(32),
                    G4 VARCHAR(32),
                    G5 VARCHAR(32),
                    G6 VARCHAR(32)
                );
            """)
        
        n_batch = len(data) // batch_size + int(len(data) / batch_size > len(data) // batch_size)

        for ibatch in range(n_batch):

            pp = f"INSERT INTO {e1}_{e2} (env, crystal, ind, shift, err, nei_ind, nei_shift, nei_err, G2, G3, G4, G5, G6)\nVALUES\n"

            for env, crystal, i, cs, err, inei, csnei, errnei, h0, h1, h2, h3, h4 in data[ibatch*batch_size:(ibatch+1)*batch_size]:

                pp += f"('{env}', '{crystal}', {i}, {cs}, {err}, {inei}, {csnei}, {errnei}, '{h0}', '{h1}', '{h2}', '{h3}', '{h4}'),\n"

            pp = pp[:-2] + ";"

            with con2:
                con2.execute(pp)

        con2.commit()
        con2.close()

Creating H...
Creating H-H...
Creating H-C...
Creating H-N...
Creating H-O...
Creating C...
Creating C-H...
Creating C-C...
Creating C-N...
Creating C-O...
Creating N...
Creating N-H...
Creating N-C...
Creating N-N...
Creating N-O...
Creating O...
Creating O-H...
Creating O-C...
Creating O-N...
Creating O-O...


In [5]:
con.close()